# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import json
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_data = get_files('data/song_data/')

In [5]:
filepath = song_data[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARPFHN61187FB575F6,41.88415,-87.63241,"Chicago, IL",Lupe Fiasco,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),279.97995,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')

In [17]:
filepath = log_files[0]

In [18]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Jordan,F,3,Hicks,259.26485,free,"Salinas, CA",PUT,NextSong,1.540009e+12,814,Supermassive Black Hole [Phones Control Voltag...,200,1543190563796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
1,None,Logged In,Jordan,F,4,Hicks,NaN,free,"Salinas, CA",PUT,Logout,1.540009e+12,814,None,307,1543190564796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
2,None,Logged Out,None,None,5,None,NaN,free,None,GET,Home,NaN,814,None,200,1543190570796,None,
3,None,Logged Out,None,None,6,None,NaN,free,None,PUT,Login,NaN,814,None,307,1543190571796,None,
4,None,Logged In,Jordan,F,7,Hicks,NaN,free,"Salinas, CA",GET,Home,1.540009e+12,814,None,200,1543190601796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df.loc[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Jordan,F,3,Hicks,259.26485,free,"Salinas, CA",PUT,NextSong,1.540009e+12,814,Supermassive Black Hole [Phones Control Voltag...,200,1543190563796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
6,Mobin Master Feat. Robin S.,Logged In,Jayden,M,1,Fox,428.64281,free,"New Orleans-Metairie, LA",PUT,NextSong,1.541034e+12,842,Show Me Love,200,1543193241796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
7,Duncan Dhu,Logged In,Bronson,M,0,Harris,211.90485,free,"Eugene, OR",PUT,NextSong,1.540907e+12,827,Rozando La Eternidad,200,1543193525796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",33
8,New Found Glory,Logged In,Bronson,M,1,Harris,222.69342,free,"Eugene, OR",PUT,NextSong,1.540907e+12,827,the king of wishful thinking,200,1543193736796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",33
9,Reel Big Fish,Logged In,Bronson,M,2,Harris,210.65098,free,"Eugene, OR",PUT,NextSong,1.540907e+12,827,Beer,200,1543193958796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",33


In [20]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-26 00:02:43.796
6   2018-11-26 00:47:21.796
7   2018-11-26 00:52:05.796
8   2018-11-26 00:55:36.796
9   2018-11-26 00:59:18.796
Name: ts, dtype: datetime64[ns]

In [21]:
time_data = [df.ts.values, t.dt.hour.values, t.dt.day.values, t.dt.isocalendar().week.values, t.dt.month.values, t.dt.year.values, t.dt.weekday.values]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [22]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543190563796,0,26,48,11,2018,0
1,1543193241796,0,26,48,11,2018,0
2,1543193525796,0,26,48,11,2018,0
3,1543193736796,0,26,48,11,2018,0
4,1543193958796,0,26,48,11,2018,0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,37,Jordan,Hicks,F,free
6,101,Jayden,Fox,M,free
7,33,Bronson,Harris,M,free
8,33,Bronson,Harris,M,free
9,33,Bronson,Harris,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [28]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.